# Model Training

In [4]:
# Import necessary libraries
import sagemaker
from sagemaker import get_execution_role
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Define the S3 bucket and prefix for storing data
bucket = sagemaker_session.default_bucket()  # Default bucket created by SageMaker
prefix = 'emp-bonus-linearlearner'  # Prefix for organizing data in S3

print(f"Using bucket: {bucket}")

Using bucket: sagemaker-us-east-1-891377137040


In [5]:
# Load the dataset 

df = pd.read_csv(f's3://{bucket}/output/data-processed/transformed_data.csv')
print(df.head())


   id      name   age   salary   hire_date department   bonus  \
0   1  Name_103  77.0  60000.0  2023-09-13  Marketing  3149.0   
1   2  Name_436  62.0  50000.0  2021-03-15  Marketing  3799.0   
2   3  Name_861  61.0  60000.0  2020-10-07         HR  9852.0   
3   4  Name_271  36.0  70000.0         NaN    Unknown  2583.0   
4   5  Name_107  78.0  60000.0  2018-12-23         IT  7194.0   

              address         phone                  email  address_length  \
0   Street 10, City 3  6.074290e+09  email_358@example.com              17   
1                 NaN           NaN                    NaN               3   
2                 NaN           NaN                    NaN               3   
3  Street 58, City 14  2.963623e+09  email_412@example.com              18   
4  Street 62, City 44  3.609542e+09  email_514@example.com              18   

  salary_category    age_group  
0          medium  Experienced  
1             low  Experienced  
2          medium  Experienced  
3       

In [6]:
# Separate features and target
X = df.drop(columns=['bonus'])  # Drop the target column from the features
y = df['bonus']

# Split the data into train (70%), validation (20%), and test (10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

Training set: 14000 samples
Validation set: 4020 samples
Test set: 1980 samples


In [7]:
# Save the datasets to CSV
train_data = pd.concat([y_train, X_train], axis=1)
val_data = pd.concat([y_val, X_val], axis=1)
test_data = pd.concat([y_test, X_test], axis=1)

# Save locally
os.makedirs('data', exist_ok=True)
train_data.to_csv('data/train.csv', index=False, header=False)
val_data.to_csv('data/validation.csv', index=False, header=False)
test_data.to_csv('data/test.csv', index=False)

# Upload to S3
train_path = sagemaker_session.upload_data('data/train.csv', bucket=bucket, key_prefix=f'{prefix}/train')
val_path = sagemaker_session.upload_data('data/validation.csv', bucket=bucket, key_prefix=f'{prefix}/validation')
test_path = sagemaker_session.upload_data('data/test.csv', bucket=bucket, key_prefix=f'{prefix}/test')

print(f"Train data uploaded to: {train_path}")
print(f"Validation data uploaded to: {val_path}")

Train data uploaded to: s3://sagemaker-us-east-1-891377137040/emp-bonus-linearlearner/train/train.csv
Validation data uploaded to: s3://sagemaker-us-east-1-891377137040/emp-bonus-linearlearner/validation/validation.csv


In [8]:
from sagemaker.estimator import Estimator

# Specify the container image for Linear Learner (built-in algorithm)
linear_learner_container = sagemaker.image_uris.retrieve('linear-learner', sagemaker_session.boto_region_name)

# Define the LinearLearner estimator
linear_estimator = Estimator(
    image_uri=linear_learner_container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{bucket}/{prefix}/output',
    sagemaker_session=sagemaker_session
)

# Set hyperparameters for Linear Learner
linear_estimator.set_hyperparameters(
    predictor_type='regressor',  # We're solving a regression problem
    mini_batch_size=32,
    epochs=10
)

In [9]:
from sagemaker.inputs import TrainingInput

# Specify the input data channels
train_input = TrainingInput(s3_data=train_path, content_type='text/csv')
val_input = TrainingInput(s3_data=val_path, content_type='text/csv')

# Train the model
linear_estimator.fit({'train': train_input, 'validation': val_input})

INFO:sagemaker:Creating training-job with name: linear-learner-2025-07-26-21-38-33-429


2025-07-26 21:38:34 Starting - Starting the training job...
2025-07-26 21:38:49 Starting - Preparing the instances for training...
2025-07-26 21:39:11 Downloading - Downloading input data...
2025-07-26 21:39:57 Downloading - Downloading the training image........Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/26/2025 21:41:17 INFO 139903893272384] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss': 'auto', 'margin': '1.0', 'quantile': '0.5', 'loss_insensitivity': '0.01', 'huber_delta':

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│   5 val_input = TrainingInput(s3_data=val_path, content_type='text/csv')                         │
│   6                                                                                              │
│   7 # Train the model                                                                            │
│ ❱ 8 linear_estimator.fit({'train': train_input, 'validation': val_input})                        │
│   9                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py:167 in wrapper  │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py:138 in wrapper  │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/estimator

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json

# Prepare payload in CSV format
payload = "\n".join(",".join(map(str, row)) for row in X_val.values)

# Send the payload to the endpoint
response = predictor.predict(payload, initial_args={"ContentType": "text/csv"})

# Extract predictions from the response
predicted_bonus = np.array([float(pred['score']) for pred in json.loads(response.decode('utf-8'))['predictions']])

# Visualization: Actual vs Predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_val, predicted_bonus, alpha=0.6, label="Predicted vs Actual")
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', label="Perfect Fit")
plt.xlabel("Actual Bonus")
plt.ylabel("Predicted Bonus")
plt.title("Actual vs. Predicted Bonus")
plt.legend()
plt.grid(True)
plt.show()

# Visualization: Residual Plot
residuals = y_val - predicted_bonus
plt.figure(figsize=(10, 6))
plt.scatter(y_val, residuals, alpha=0.6)
plt.axhline(0, color='r', linestyle='--', label="Zero Error Line")
plt.xlabel("Actual Bonus")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.legend()
plt.grid(True)
plt.show()